In [1]:
# WILL STRESS TEST: FITTED Y* & H0 COMPETITION (RESONANCE INTERFERENCE VERSION)
# Updated to use the new Interference Law: V_tot^2 = V_bar^2 + sqrt(V_bar^2 * a * r)

import numpy as np
import pandas as pd
import requests, io
from scipy.optimize import minimize_scalar
from sklearn.metrics import mean_squared_error, r2_score

# -----------------------------
# SETTINGS
# -----------------------------
H0_LIST = [65.5, 67.4, 73.2]
MODELS = [
    {"label": "WILL (3π)", "denom": 3 * np.pi},
    {"label": "Geom (2π)", "denom": 2 * np.pi},
    {"label": "Geom (1π)", "denom": 1 * np.pi},
    {"label": "Geom (4π)", "denom": 4 * np.pi},
    {"label": "Geom (6π)", "denom": 6 * np.pi}
]

# Physics Constants
C_KMS = 299792.458
MPC_TO_M = 3.08567758e22
KPC_TO_M = 3.08567758e19

# Data URLs
URL_T1 = "https://raw.githubusercontent.com/AntonRize/WILL/refs/heads/main/SPARC%20DATA/table1.dat"
URL_T2 = "https://raw.githubusercontent.com/AntonRize/WILL/refs/heads/main/SPARC%20DATA/table2.dat"

# -----------------------------
# CORE FUNCTIONS
# -----------------------------
def load_data():
    print("Downloading SPARC database...")
    try:
        r1 = requests.get(URL_T1); r1.raise_for_status()
        t1 = pd.read_fwf(io.BytesIO(r1.content), colspecs=[(0,11), (29,34), (113,114)], header=None, names=["Name", "Inc", "Qual"])

        r2 = requests.get(URL_T2); r2.raise_for_status()
        t2 = pd.read_fwf(io.BytesIO(r2.content), colspecs=[(0,11), (19,25), (26,32), (33,38), (39,45), (46,52), (53,59)], header=None, names=["Name", "Rad", "Vobs", "e_Vobs", "Vgas", "Vdisk", "Vbul"])

        t1["Name"] = t1["Name"].astype(str).str.strip()
        t2["Name"] = t2["Name"].astype(str).str.strip()

        t1["Inc"] = pd.to_numeric(t1["Inc"], errors="coerce")
        t2["Rad"] = pd.to_numeric(t2["Rad"], errors="coerce")
        t2["Vobs"] = pd.to_numeric(t2["Vobs"], errors="coerce")

        t2 = t2.dropna(subset=["Name", "Rad", "Vobs"])
        t2 = t2[(t2["Rad"] > 0) & (t2["Vobs"] > 0)]

        print(f"Table 1 loaded: {len(t1)} rows.")
        print(f"Table 2 loaded: {len(t2)} rows.")
        return t1, t2
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None

def calculate_a_mach(h0, denom):
    h0_si = (h0 * 1000) / MPC_TO_M
    return (C_KMS * 1000 * h0_si) / denom

# --- NEW PHYSICS ENGINE: INTERFERENCE LAW ---
def predict_velocity(y_star, v_gas, v_disk, v_bul, rad_kpc, a_mach):
    # 1. Baryonic V^2 (km/s)^2
    v_bar_sq = np.abs(v_gas)*v_gas + y_star*(np.abs(v_disk)*v_disk + np.abs(v_bul)*v_bul)
    v_bar_sq = np.maximum(v_bar_sq, 0)

    # 2. Convert to SI for Interference Calculation
    v_bar_sq_si = v_bar_sq * 1e6     # (m/s)^2
    rad_m = rad_kpc * KPC_TO_M       # meters

    # 3. Calculate Interference Term (Vacuum Energy)
    # V_vac^2 = sqrt( V_bar^2 * Work_Global )
    # Work_Global = a_Mach * r
    work_global = a_mach * rad_m
    v_vac_sq_si = np.sqrt(v_bar_sq_si * work_global)

    # 4. Total Energy
    v_tot_sq_si = v_bar_sq_si + v_vac_sq_si

    # 5. Return velocity in km/s
    return np.sqrt(v_tot_sq_si) / 1000.0

def fit_galaxy_y_star(galaxy_data, a_mach):
    rad = galaxy_data["Rad"].values
    v_obs = galaxy_data["Vobs"].values
    e_obs = pd.to_numeric(galaxy_data["e_Vobs"], errors='coerce').fillna(10.0).values
    e_obs = np.maximum(e_obs, 5.0)

    v_gas = pd.to_numeric(galaxy_data["Vgas"], errors='coerce').fillna(0).values
    v_disk = pd.to_numeric(galaxy_data["Vdisk"], errors='coerce').fillna(0).values
    v_bul = pd.to_numeric(galaxy_data["Vbul"], errors='coerce').fillna(0).values

    def objective(y_val):
        v_pred = predict_velocity(y_val, v_gas, v_disk, v_bul, rad, a_mach)
        return np.sum(((v_obs - v_pred) / e_obs)**2)

    # Updated bounds to (0.05, 5.0) to match text description of "physically realistic bounds"
    res = minimize_scalar(objective, bounds=(0.05, 5.0), method='bounded')
    best_y = res.x

    # Return best vectors for global stats
    # Recalculate baryonic acceleration for final output
    v_bar_sq = np.abs(v_gas)*v_gas + best_y*(np.abs(v_disk)*v_disk + np.abs(v_bul)*v_bul)
    v_bar_sq = np.maximum(v_bar_sq, 1e-5)

    conv = (1000.0**2) / KPC_TO_M
    g_bar = (v_bar_sq / rad) * conv     # m/s^2
    g_obs = ((v_obs**2) / rad) * conv   # m/s^2

    return g_bar, g_obs, best_y

# -----------------------------
# MAIN EXECUTION
# -----------------------------
t1, t2 = load_data()

if t1 is not None:
    hq_galaxies = []
    print("\nFiltering galaxies...")
    for gal in t1["Name"].unique():
        props = t1[t1["Name"] == gal]
        if len(props) == 0: continue
        inc = props.iloc[0]["Inc"]
        if not np.isnan(inc) and inc >= 30:
            hq_galaxies.append(gal)

    print(f"Number of galaxies passing Inclination filter (>30 deg): {len(hq_galaxies)}")

    if len(hq_galaxies) == 0:
        print("CRITICAL ERROR: No galaxies passed filter.")
    else:
        for h0 in H0_LIST:
            print(f"\n=========================================================================================")
            print(f" H0 = {h0} km/s/Mpc")
            print(f"=========================================================================================")
            print(f"{'MODEL':<15} | {'a_Mach (m/s2)':<15} | {'RMSE (dex)':<12} | {'R² Score':<10} | {'Mean Y*':<8}")
            print("-" * 90)

            for m in MODELS:
                a_mach = calculate_a_mach(h0, m["denom"])
                all_g_bar = []
                all_g_obs = []
                y_stars = []

                for gal in hq_galaxies:
                    gdata = t2[t2["Name"] == gal]
                    if len(gdata) < 5: continue
                    gb, go, y_best = fit_galaxy_y_star(gdata, a_mach)

                    mask = (gb > 1e-14) & (go > 1e-14)
                    all_g_bar.extend(gb[mask])
                    all_g_obs.extend(go[mask])
                    y_stars.append(y_best)

                if len(all_g_bar) == 0:
                    continue

                all_g_bar = np.array(all_g_bar)
                all_g_obs = np.array(all_g_obs)

                # --- GLOBAL STATS WITH INTERFERENCE LAW ---
                # g_tot = g_bar + sqrt(g_bar * a_mach)
                # This corresponds to V^2 = V_b^2 + sqrt(V_b^2 * a * r)
                g_pred = all_g_bar + np.sqrt(all_g_bar * a_mach)

                lg_obs = np.log10(all_g_obs)
                lg_pred = np.log10(g_pred)

                rmse = np.sqrt(mean_squared_error(lg_obs, lg_pred))
                r2 = r2_score(lg_obs, lg_pred)
                mean_y = np.mean(y_stars)

                print(f"{m['label']:<15} | {a_mach:.2e}        | {rmse:.5f}      | {r2:.5f}     | {mean_y:.2f}")

        print("\nDone. Verify if 3pi is still the winner.")

Table 1 loaded: 175 rows.
Table 2 loaded: 3391 rows.

Filtering galaxies...
Number of galaxies passing Inclination filter (>30 deg): 163

 H0 = 65.5 km/s/Mpc
MODEL           | a_Mach (m/s2)   | RMSE (dex)   | R² Score   | Mean Y* 
------------------------------------------------------------------------------------------
WILL (3π)       | 6.75e-11        | 0.13415      | 0.94387     | 0.60
Geom (2π)       | 1.01e-10        | 0.13964      | 0.93919     | 0.45
Geom (1π)       | 2.03e-10        | 0.16483      | 0.91509     | 0.26
Geom (4π)       | 5.06e-11        | 0.13509      | 0.94312     | 0.72
Geom (6π)       | 3.38e-11        | 0.13988      | 0.93912     | 0.92

 H0 = 67.4 km/s/Mpc
MODEL           | a_Mach (m/s2)   | RMSE (dex)   | R² Score   | Mean Y* 
------------------------------------------------------------------------------------------
WILL (3π)       | 6.95e-11        | 0.13431      | 0.94374     | 0.59
Geom (2π)       | 1.04e-10        | 0.14033      | 0.93858     | 0.44
Geo